In [65]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [317]:
#Reading in all data, tab delimited
playByPlayDf = pd.read_csv('NBA Hackathon - Play by Play Data Sample (50 Games).txt',sep='\t')
lineupsDf = pd.read_csv('NBA Hackathon - Game Lineup Data Sample (50 Games).txt',sep='\t')
codesDf = pd.read_csv('NBA Hackathon - Event Codes.txt',sep='\t')

In [318]:
def find_code(msg_type,action_type=None):
    if action_type:
        return codesDf[(codesDf['Event_Msg_Type'] == msg_type) & (codesDf['Action_Type']==action_type)]
    else:
        return codesDf[codesDf['Event_Msg_Type'] == msg_type]

In [319]:
codesDf.Event_Msg_Type.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

##### Code descriptions
1: Made Shot <br>
2: Missed Shot <br>
3: Free Throw (Except Action Type == 0)<br>
6: Foul<br>
8: Substitution<br>

###### Sort order for play by play per instructions
Period (ascending), PC_Time (descending), WC_Time (ascending),
Event_Number (ascending)

In [320]:
print(lineupsDf.columns)
print('')
print(playByPlayDf.columns)

Index(['Game_id', 'Period', 'Person_id', 'Team_id', 'status'], dtype='object')

Index(['Game_id', 'Event_Num', 'Event_Msg_Type', 'Period', 'WC_Time',
       'PC_Time', 'Action_Type', 'Option1', 'Option2', 'Option3', 'Team_id',
       'Person1', 'Person2', 'Team_id_type'],
      dtype='object')


In [321]:
playByPlayDf.sort_values('Period',ascending=True,inplace=True)
playByPlayDf.sort_values('PC_Time',ascending=False,inplace=True)
playByPlayDf.sort_values('WC_Time',ascending=True,inplace=True)
playByPlayDf.sort_values('Event_Num',ascending=True,inplace=True)
playByPlayDf.reset_index(inplace=True)
playByPlayDf.drop(axis=0,columns=['index'],inplace=True)

In [322]:
freeThrows = playByPlayDf[playByPlayDf.Event_Msg_Type == 3].index

In [326]:
pbpArray = np.array(playByPlayDf)
for idx in freeThrows:
    prevIdx = idx - 1
    while pbpArray[prevIdx,2] == 8:
        pbpArray[[prevIdx,idx]] = pbpArray[[idx,prevIdx]]
        prevIdx -= 1

In [364]:
subSwap = pd.DataFrame(pbpArray,columns = playByPlayDf.columns)

In [365]:
subSwap.loc[(playByPlayDf.Event_Msg_Type == 3) & (playByPlayDf.Option1 == 2),'Option1'] = 0

In [366]:
subSwap['onthefloor'] = ''
stacked = pd.DataFrame()
for gameId in lineupsDf['Game_id'].unique():
    gameLineups = lineupsDf[lineupsDf['Game_id'] == gameId]
    gamePbp = subSwap[subSwap['Game_id'] == gameId]
    gamePbp = gamePbp[gamePbp['Team_id'].isin(gameLineups['Team_id'].unique())]
    currDict = dict()
    for team in gameLineups['Team_id'].unique():
        currDict[team] = []
        currTeam = set(gameLineups[gameLineups['Team_id'] == team]['Person_id'].unique())
        lastIter=set()
        while len(currTeam) != len(lastIter):
            lastIter=deepcopy(currTeam)
            currTeam.update(
                gamePbp[
                    (gamePbp['Event_Msg_Type'] == 8) & 
                    (gamePbp['Person1'].isin(currTeam))
                ]['Person2'].unique()
            )
        teamCount = len(currTeam)
        currDf = pd.DataFrame(
            list(zip(currTeam,[0] * teamCount, [gameId] * teamCount,[team] * teamCount)),
            columns=['playerid','plus_minus','gameid','teamid']
        )
        stacked = pd.concat([stacked,currDf])
    subSwap.loc[subSwap['Game_id'] == gameId,'onthefloor'] = [currDict]

In [367]:
subSwap.columns

Index(['Game_id', 'Event_Num', 'Event_Msg_Type', 'Period', 'WC_Time',
       'PC_Time', 'Action_Type', 'Option1', 'Option2', 'Option3', 'Team_id',
       'Person1', 'Person2', 'Team_id_type', 'onthefloor'],
      dtype='object')

In [368]:
stacked.columns

Index(['playerid', 'plus_minus', 'gameid', 'teamid'], dtype='object')

In [374]:
vals

Game_id                            fdeb2950c4d5209d449ebd2d6afac11e
Event_Num                                                       346
Event_Msg_Type                                                    1
Period                                                            3
WC_Time                                                      593878
PC_Time                                                        3840
Action_Type                                                       1
Option1                                                           3
Option2                                                           0
Option3                                                           0
Team_id                            01be0ad4af7aeb1f6d2cc2b6b6d6d811
Person1                            c6e0c30c8c02b0706682ec89eb6fc565
Person2                            07695ef9feebc6ba02831f0f3c22662a
Team_id_type                                                      2
onthefloor        {'3cd0b15957ceb80f5125bef8bd1b

In [427]:
subSwapNew = subSwap.reset_index().copy()
prevPeriod = -1
prevTeamDict = dict()
count = 1
plusMinus = stacked.copy()
for gameId in subSwapNew.Game_id.unique():
    print(f'Finding data for game {count} out of {len(subSwapNew.Game_id.unique())}')
    count+=1
    currGame = subSwapNew[subSwapNew.Game_id == gameId]
    for idx, vals in currGame.iterrows():
        if vals.Period != prevPeriod:
            prevPeriod = vals.Period
            for team in vals.onthefloor.keys():
                subSwapNew.iloc[vals['index']].onthefloor[team] = list(
                    lineupsDf[
                        (lineupsDf['Game_id'] == gameId) & 
                        (lineupsDf['Team_id'] == team) & 
                        (lineupsDf['Period'] == vals.Period)

                    ].Person_id
                )
        elif vals.Event_Msg_Type == 8:
            for team in vals.onthefloor.keys():
                if vals.Person1 in vals.onthefloor[team]:
                    playerIdx = subSwapNew.iloc[vals['index']].onthefloor[team].index(vals.Person1)
                    subSwapNew.iloc[vals['index']].onthefloor[team][playerIdx] = vals.Person2
        else:
            subSwapNew.loc[vals['index'],'onthefloor'] = [prevTeamDict]
        
        prevTeamDict = subSwapNew.iloc[vals['index']].onthefloor
        
        if vals.Event_Msg_Type in [1,3]:
            points = vals.Option1
            scoringTeam = vals.Team_id
            for team in vals.onthefloor.keys():
                if len(vals.onthefloor[team]) != 5:
                    print(vals.onthefloor[team])
                for player in vals.onthefloor[team]:
                    if team == scoringTeam:
                        plusMinus.loc[
                            (plusMinus.playerid == player) & (plusMinus.gameid == vals.Game_id),
                            'plus_minus'] += points
                    else:
                        plusMinus.loc[
                            (plusMinus.playerid == player) & (plusMinus.gameid == vals.Game_id),
                            'plus_minus'] -= points
            if plusMinus.plus_minus.sum() != 0:
                print('ERROR')
        
        for team in prevTeamDict.keys():
            if len(prevTeamDict[team]) != 5:
                print (vals['index'],gameId,prevTeamDict)

Finding data for game 1 out of 50
Finding data for game 2 out of 50
Finding data for game 3 out of 50
Finding data for game 4 out of 50
Finding data for game 5 out of 50
Finding data for game 6 out of 50
Finding data for game 7 out of 50
Finding data for game 8 out of 50
Finding data for game 9 out of 50
Finding data for game 10 out of 50
Finding data for game 11 out of 50
Finding data for game 12 out of 50
Finding data for game 13 out of 50
Finding data for game 14 out of 50
Finding data for game 15 out of 50
Finding data for game 16 out of 50
Finding data for game 17 out of 50
Finding data for game 18 out of 50
Finding data for game 19 out of 50
Finding data for game 20 out of 50
Finding data for game 21 out of 50
Finding data for game 22 out of 50
Finding data for game 23 out of 50
Finding data for game 24 out of 50
Finding data for game 25 out of 50
Finding data for game 26 out of 50
Finding data for game 27 out of 50
Finding data for game 28 out of 50
Finding data for game 29 out 

In [431]:
finalAnswer = plusMinus.rename(columns={'playerid':'Player_ID','plus_minus':'Player_Plus/Minus','gameid':'Game_ID'}).drop(columns='teamid')

In [433]:
finalAnswer = finalAnswer[['Game_ID','Player_ID','Player_Plus/Minus']]

In [ ]:
finalAnswer.to_csv('Q1_BBALL.csv')